In [ ]:
import cv2
import numpy as np
from pathlib import Path
from PIL import Image
import gradio as gr
from skimage.metrics import mean_squared_error, structural_similarity as ssim


class MosaicMaker:
    def __init__(self, tile_dir: str):
        """
        Load tile images from the specified directory.
        Supports multi-size tiles (32x32, 16x16, 8x8).
        """
        self.tiles, self.tile_colors = self._load_tiles(tile_dir)
    

    # Step 1: Image Selection and Preprocessing
    def _load_tiles(self, tile_dir: str):
        """
        Load tile images from the specified directory.
        Resize them to 32x32, 16x16, and 8x8.
        Compute and store their average colors.
        Returns: (tiles, tile_colors)
        - tiles: dict of size to list of tile images (numpy arrays)
        - tile_colors: dict of size to list of average colors (numpy arrays)
        """
        tiles = {32: [], 16: [], 8: []}
        tile_colors = {32: [], 16: [], 8: []}
        valid_exts = {".jpg", ".jpeg", ".png", ".webp", ".bmp"}
    
        for path in Path(tile_dir).rglob("*"):
            if path.suffix.lower() not in valid_exts:
                continue
            tile_bgr = cv2.imread(str(path))
            if tile_bgr is None:
                continue
            tile = cv2.cvtColor(tile_bgr, cv2.COLOR_BGR2RGB)
    
            for size in [32, 16, 8]:
                resized = self.resize_with_crop(tile, (size, size))
                tiles[size].append(resized)
                tile_colors[size].append(resized.mean(axis=(0, 1)))
    
        # convert to numpy
        for size in tiles:
            tile_colors[size] = np.asarray(tile_colors[size], dtype=np.float32)
    
        return tiles, tile_colors


    # Step 2: Image Griding and Thresholding
    def color_quantization(self, img_rgb: np.ndarray, k: int = 16) -> np.ndarray:
        """
        Perform color quantization using K-means and return the quantized image (keep RGB).
        - img_rgb: input image in RGB format (numpy array)
        - k: number of colors
        Returns: quantized image (numpy array)
        """
        Z = img_rgb.reshape((-1, 3)).astype(np.float32)
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 20, 1.0)
        _compactness, labels, centers = cv2.kmeans(
            Z, k, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS
        )
        centers = centers.astype(np.uint8)
        res = centers[labels.flatten()]
        return res.reshape(img_rgb.shape)


    def resize_with_crop(self, img_rgb: np.ndarray, target_size=(512, 512)) -> np.ndarray:
        """
        Resize the image to cover the target_size while maintaining aspect ratio,
        then center-crop to the fixed resolution.
        - img_rgb: input image in RGB format (numpy array)
        - target_size: (height, width) tuple
        Returns: resized and cropped image (numpy array)
        """
        th, tw = target_size
        h, w = img_rgb.shape[:2]
        # Determine scale to cover target size
        scale = max(th / h, tw / w)
        new_w = max(1, int(round(w * scale)))
        new_h = max(1, int(round(h * scale)))
        resized = cv2.resize(img_rgb, (new_w, new_h), interpolation=cv2.INTER_AREA)

        x0 = max(0, (new_w - tw) // 2)
        y0 = max(0, (new_h - th) // 2)
        cropped = resized[y0:y0 + th, x0:x0 + tw]
        return cropped

    
    def grid_mean_colors(self, img_rgb: np.ndarray):
        """
        Divide the image into adaptive blocks (32x32, 16x16, 8x8) based on texture variance.
        Return a list of blocks: (y1, y2, x1, x2, size, mean_color)
        - img_rgb: input image in RGB format (numpy array)
        Returns: list of blocks
        """
        H, W = img_rgb.shape[:2]
        blocks = []
    
        # texture variance thresholds
        var_thresh = {32: 80.0, 16: 60.0}
    
        def block_variance(patch):
            # use mean of per-channel variance
            return float(np.var(patch, axis=(0, 1)).mean())

        # first try 32x32 blocks
        for y in range(0, H, 32):
            for x in range(0, W, 32):
                y2 = min(y + 32, H)
                x2 = min(x + 32, W)
                patch = img_rgb[y:y2, x:x2]
                v = block_variance(patch)
                if v < var_thresh[32] and (y2 - y == 32 and x2 - x == 32):
                    mean_color = patch.mean(axis=(0, 1))
                    blocks.append((y, y2, x, x2, 32, mean_color))
                else:
                    # further split into 16x16
                    for yy in range(y, y2, 16):
                        for xx in range(x, x2, 16):
                            yy2 = min(yy + 16, H)
                            xx2 = min(xx + 16, W)
                            patch16 = img_rgb[yy:yy2, xx:xx2]
                            v16 = block_variance(patch16)
                            if v16 < var_thresh[16] and (yy2 - yy == 16 and xx2 - xx == 16):
                                mean_color = patch16.mean(axis=(0, 1))
                                blocks.append((yy, yy2, xx, xx2, 16, mean_color))
                            else:
                                # finally split into 8x8
                                for y8 in range(yy, yy2, 8):
                                    for x8 in range(xx, xx2, 8):
                                        y82 = min(y8 + 8, H)
                                        x82 = min(x8 + 8, W)
                                        patch8 = img_rgb[y8:y82, x8:x82]
                                        mean_color = patch8.mean(axis=(0, 1))
                                        # no variance check for 8x8, just take it
                                        if (y82 - y8 == 8 and x82 - x8 == 8):
                                            blocks.append((y8, y82, x8, x82, 8, mean_color))
    
        return blocks


    # Step 3: Tile Mapping
    def build_mosaic(self, blocks) -> np.ndarray:
        """
        Build the mosaic image from blocks and tiles.
        - blocks: list of (y1, y2, x1, x2, size, mean_color)
        Returns: mosaic image (numpy array)
        """
        # calculate output size
        max_y = max(b[1] for b in blocks)
        max_x = max(b[3] for b in blocks)
        out = np.zeros((max_y, max_x, 3), dtype=np.uint8)

        # check if tiles is dict (multi-size) or list (single-size)
        tiles_is_dict = isinstance(self.tiles, dict)
    
        for (y1, y2, x1, x2, size, mean_color) in blocks:
            mean_color = np.asarray(mean_color, dtype=np.float32)
    
            if tiles_is_dict:
                # multi-size version: 32,16,8
                tile_bank = self.tiles[size]
                color_bank = self.tile_colors[size]  # shape: (N,3)
                # find the closest tile by color
                dists = np.linalg.norm(color_bank - mean_color, axis=1)
                idx = int(np.argmin(dists))
                tile = tile_bank[idx]  # ndarray (size,size,3)
            else:
                # single size version: resize on the fly
                color_bank = self.tile_colors  # ndarray (N,3)
                dists = np.linalg.norm(color_bank - mean_color, axis=1)
                idx = int(np.argmin(dists))
                base_tile = self.tiles[idx]
                tile = cv2.resize(base_tile, (size, size), interpolation=cv2.INTER_AREA)
    
            out[y1:y2, x1:x2] = tile
    
        return out


    def blend_mosaic_with_ref(
        self,
        mosaic_img: np.ndarray,
        ref_img: np.ndarray,
        blocks: list,
        alpha: float = 0.5
    ) -> np.ndarray:
        """
        Blend the mosaic and reference image cell by cell.
        Works with adaptive blocks.
        - mosaic_img: generated mosaic image (numpy array)
        - ref_img: reference image to blend with (numpy array)
        - blocks: list of (y1, y2, x1, x2,
                    size, mean_color)
        - alpha: blending factor (0.0: only mosaic, 1.0: only ref)
        Returns: blended image (numpy array)
        """
        if mosaic_img.shape != ref_img.shape:
            raise ValueError("mosaic_img and ref_img must have the same shape.")
    
        out = mosaic_img.copy()
        a = float(alpha)
        if a <= 0:
            return out
        if a >= 1:
            return ref_img.copy()
    
        for (y1, y2, x1, x2, size, mean_color) in blocks:
            tile = out[y1:y2, x1:x2].astype(np.float32)
            cell = ref_img[y1:y2, x1:x2].astype(np.float32)
            blended = (1.0 - a) * tile + a * cell
            out[y1:y2, x1:x2] = np.clip(blended, 0, 255).astype(np.uint8)
    
        return out


    def evaluate_performance(self, orig_img, mosaic_img):
        """
        Evaluate the mosaic quality using MSE and SSIM.
        - orig_img: original image (numpy array)
        - mosaic_img: generated mosaic image (numpy array)
        Returns: (mse, ssim)
        """
        # secure they have the same shape
        assert orig_img.shape == mosaic_img.shape
        
        # MSE
        mse_val = mean_squared_error(orig_img, mosaic_img)
        
        # SSIM
        ssim_val, _ = ssim(orig_img, mosaic_img, channel_axis=2, full=True)
        
        return mse_val, ssim_val


# Step 4: Building the Gradio Interface
with gr.Blocks(title="Image Tile Mosaic") as demo:
    gr.Markdown("## Upload an image and generate a mosaic.")
    with gr.Row():
        inp_img = gr.Image(type="pil", label="Input Image")
        out_img = gr.Image(type="pil", label="Output Mosaic")
    with gr.Row():
        blend = gr.Slider(0.0, 0.8, value=0.4, step=0.05, label="Blend with original (per cell)")
    btn = gr.Button("Generate")
    metrics = gr.Textbox(label="Quality metrics", interactive=False)

    def _run(pil_img: Image.Image, blend: float):
        # 1) PIL to NumPy
        img = np.array(pil_img.convert("RGB"))

        # 2) Resize to a fixed size (recommended 512x512 for easy division)
        target = (512, 512)
        maker = MosaicMaker("tiles_folder")
        img_512 = maker.resize_with_crop(img, target)

        # 3) Color Quantization
        quant = maker.color_quantization(img_512, k=16)

        blocks = maker.grid_mean_colors(quant)

        # 4) Generate Mosaic
        mosaic = maker.build_mosaic(blocks)

        # 5) Blend with original if needed
        final = maker.blend_mosaic_with_ref(mosaic, quant, blocks, alpha=float(blend))

        # 6) Evaluate performance
        mse_val, ssim_val = maker.evaluate_performance(img_512, final)
    
        metrics_text = f"MSE: {mse_val:.2f}   SSIM: {ssim_val:.4f}"

        return Image.fromarray(final), metrics_text

    btn.click(_run, inputs=[inp_img, blend], outputs=[out_img, metrics])

    gr.Examples(
        examples=[
            ["MonaLisaCat.jpg", 0.0],
            ["ResponseCat.png", 0.3],
            ["ShockedCat.png", 0.6],
        ],
        inputs=[inp_img, blend],
        outputs=[out_img, metrics],        # clicking an example will auto-fill inputs and run the function
        fn=_run,
        label="Try these examples",
        cache_examples=True
    )
    
    demo.launch()



* Running on local URL:  http://127.0.0.1:7896
Caching examples at: '/Users/shangjiuyue/Desktop/NEU/5330/HW1/HW1/.gradio/cached_examples/424'
* To create a public link, set `share=True` in `launch()`.
